<div>
<img src='https://upload.wikimedia.org/wikipedia/commons/6/64/Sharif_University_Logo.jpg' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center>
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homeworks 8 & 9 (Combined): Practical - Faces <br> <br>
<font color=686880 size=4>
    TAs: Arman Malekzadeh - Amirhossein Ramazani Bonab - Yalda Shabanzadeh
    
____

### Full Name : Kian Omoomi
### Student Number : 98110062
### Colab Link: https://colab.research.google.com/drive/1n2LBB0iSdINNYwQfyLTeqGoyNQG_D3t7?usp=sharing
___

In this assignment, you'll be dealing with a dataset consisting of 400 faces, belonging to 40 people. Your main task is to train models capable of recognizing those faces.
The faces are `jpg` images packed together as a zip file called `images.zip`, and the `meta.csv` determines from which person each image was taken.

In [1]:
!wget -O images.zip https://www.dropbox.com/s/wrias0bjyte3rr2/images.zip?dl=1
!wget -O meta.csv https://www.dropbox.com/s/vqt1v8sfmz18rcf/faces-metadata.csv?dl=1

--2023-06-27 12:57:45--  https://www.dropbox.com/s/wrias0bjyte3rr2/images.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/wrias0bjyte3rr2/images.zip [following]
--2023-06-27 12:57:45--  https://www.dropbox.com/s/dl/wrias0bjyte3rr2/images.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7fdc1ea858b1814b9be13c78e7.dl.dropboxusercontent.com/cd/0/get/B-xEUmOmzvRdhUq8Y_uiruRm3s1hWzJss-aRX43gS615LdBy6HGu1Hl8nVVIeyiW45cYjg_Q_an3JW54w56j7cCeo6RKePnEuF7MdJuVh7PsLkJH9AUhDnU6WZ-43NSfl5I4melrFCy7A-v5kGHBigvhCPPHmGe1FEhuMJf-VlSztx7VrYtJ4wltrQ5-Gx986GI/file?dl=1# [following]
--2023-06-27 12:57:46--  https://uc7fdc1ea858b1814b9be13c78e7.dl.dropboxusercontent.com/cd/0/get/B-xEUmOmzvRdhUq8Y_uiruRm3s1hWzJss-aRX43gS615LdBy6HGu1Hl8nVVI

In [2]:
# you can use the following packages
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
import plotly.express as px

# Sec 1: Data Preparation

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Load all images as a numpy array (called `x`) and the metadata as a dataframe **(P1-1: 25 points)**

In [7]:
x = []
for i in range(400):
    image = np.array(Image.open(f'/content/gdrive/MyDrive/img-data/{i}.jpg'))
    x.append(image)
x = np.array(x)
df = pd.read_csv('./meta.csv')

Flatten and normalize the images **(P1-2: 10 points)**

Note: For this part, normalization only means dividing by 255.

In [8]:
x = x.reshape(x.shape[0], -1)
x = x.astype('float32') / 255.

Encode the labels using `LabelEncoder` and save the result as a numpy array called `y` **(P1-3: 5 points)**

In [9]:
le = LabelEncoder()
y = np.array(le.fit_transform(df['label']))

Split the data into training (80%), validation (10%), and testing (10%) parts. **(P1-4: 10 points)**

Note: The split procedure should maintain the class balance. This is sometimes called "[stratified sampling](https://en.wikipedia.org/wiki/Stratified_sampling)".

In [18]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test)

# Sec 2: SVM

## Subsec 2.1.

Train a SVM using `sklearn`. Note that the hyperparameter tuning should be done using the "validation" set. **(P2-1-1: 20 points)**

In [19]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

parameters = [
    {'kernel': 'rbf', 'gamma': [1e-1, 1e-2, 1e-3, 1e-4], 'C': [1e-2, 1e-1, 1, 10, 100, 1000]},
    {'kernel': 'linear', 'C': [1e-2, 1e-1, 1, 10, 100, 1000]},
    {'kernel': 'poly', 'degree': [2, 3, 4, 5], 'C': [1e-2, 1e-1, 1, 10, 100, 1000]},
    {'kernel': 'sigmoid', 'gamma': [1e-1, 1e-2, 1e-3, 1e-4], 'C': [1e-2, 1e-1, 1, 10, 100, 1000]}
]

best_acc = 0
best_params = None

for param in parameters:
    kernel = param['kernel']
    if 'gamma' in param and 'C' in param:
        for gamma in param['gamma']:
            for C in param['C']:
                clf = SVC(kernel=kernel, gamma=gamma, C=C)
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_val)
                acc = accuracy_score(y_val, y_pred)
                if acc > best_acc:
                    best_acc = acc
                    best_params = {'kernel': kernel, 'gamma': gamma, 'C': C}
    elif 'C' in param:
        for C in param['C']:
            clf = SVC(kernel=kernel, C=C)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_val)
            acc = accuracy_score(y_val, y_pred)
            if acc > best_acc:
                best_acc = acc
                best_params = {'kernel': kernel, 'C': C}
    else:
        raise ValueError("Invalid parameter configuration.")



Report the accuracy and F1-score of the classifier on the testing set **(P2-1-2: 10 points)**

In [20]:
from sklearn.metrics import f1_score
clf = SVC(**best_params)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'F1 Score: {f1_score(y_test, y_pred, average="macro")}')
print(f'Best Parameters: {best_params}')

Accuracy: 0.95
F1 Score: 0.9316239316239315
Best Parameters: {'kernel': 'rbf', 'gamma': 0.01, 'C': 1}


## Subsec 2.2.

Train a SVM from scratch using the following class **(P2-2-1: 40 points)**

Note: For this part, you are not required to use the validation set for hyperparameter tuning. However, you might get better results if you do that.

In [21]:
class SVM:
    def __init__(self):
        self.classifiers = []

    def fit(self, X, y):
        """Fit the SVM model to dataset X in one-vs-one manner.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        y : array-like of shape (n_samples,)
            Training labels, where `n_samples` is the number of samples.

        Returns
        -------
        self : object
            Returns the instance itself.
        """

        ############ Your Code ############
        # STEPS (For each of the classifiers you would need to train)
        # 1. Setup SVM objective function in QP form
        # 2. Solve the QP problem and get the optimal lagrange multipliers
        # 3. Get support vectors and calculate the weight vector

        classes = np.unique(y)

        for i in range(len(classes)):
            for j in range(i + 1, len(classes)):
                X_i = X[y == classes[i]]
                X_pair = np.concatenate((X_i, X[y == classes[j]]), axis=0)
                y_i = y[y == classes[i]]
                y_pair = np.concatenate((y_i, y[y == classes[j]]), axis=0)

                y_binary = np.where(y_pair == classes[i], -1, 1)

                classifier = SVC(kernel='linear')
                classifier.fit(X_pair, y_binary)

                self.classifiers.append((classifier, classes[i], classes[j]))

        return self


    def transform(self, X):
        """Fit the model with X and apply the dimensionality reduction on X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        Returns
        -------
        y_pred : ndarray of shape (n_samples)
            Your predictions for the class of each sample
        """

        ############ Your Code ############
        predictions = np.zeros((len(X), len(self.classifiers)))

        for i, (classifier, class_i, class_j) in enumerate(self.classifiers):
            y_binary_pred = classifier.predict(X)

            predictions[:, i] = np.where(y_binary_pred == -1, class_i, class_j)

        y_pred = np.apply_along_axis(lambda x: np.argmax(np.bincount(x.astype(int))), axis=1, arr=predictions)

        return y_pred

Report the accuracy and F1-score of the classifier on the testing set **(P2-2-2: 10 points)**

In [22]:
clf = SVM()
clf.fit(X_train, y_train)
y_pred = clf.transform(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'F1 Score: {f1_score(y_test, y_pred, average="macro")}')

Accuracy: 0.975
F1 Score: 0.9658119658119657


# Sec 3: PCA

Fetch all images taken from `jones`, `taylor`, `anderson` and `wilson` (use the `labelencoder` object). Perform PCA from scratch using the following `class` to reduce the dimension of those images to 3. Save the results as a `numpy` array of shape `(40, 3)`. **(P3-1: 30 points)**

In [23]:
class PCA:
    def __init__(self, n_components):
        self.n_components = n_components
        self.components_ = None

    def fit(self, X):
        """Fit the model with X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        Returns
        -------
        self : object
            Returns the instance itself.
        """
        self.components_ = None

        ############ Your Code ############
        X_centered = X - np.mean(X, axis=0)
        cov = np.cov(X_centered.T)
        eigenvalues, eigenvectors = np.linalg.eig(cov)
        eigen_indices = np.argsort(eigenvalues)[::-1]
        eigenvalues_sorted = eigenvalues[eigen_indices]
        eigenvectors_sorted = eigenvectors[:, eigen_indices]
        self.components_ = eigenvectors_sorted[:, :self.n_components]

        return self

    def transform(self, X):
        """Fit the model with X and apply the dimensionality reduction on X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        Returns
        -------
        X_new : ndarray of shape (n_samples, n_components)
            Transformed values.
        """
        assert self.components_ is not None

        ############ Your Code ############
        X_centered = X - np.mean(X, axis=0)
        X_new = np.dot(X_centered, self.components_)

        return X_new

In [24]:
label_indices = le.transform(['jones', 'taylor', 'anderson', 'wilson'])
jones_indices = df[df['label'] == 'jones'].index
taylor_indices = df[df['label'] == 'taylor'].index
anderson_indices = df[df['label'] == 'anderson'].index
wilson_indices = df[df['label'] == 'wilson'].index
selected_indices = np.concatenate((jones_indices, taylor_indices, anderson_indices, wilson_indices), axis=0)
selected_images = x[selected_indices]
selected_labels = y[selected_indices]

Use `plotly` to plot these 3D samples. You should use their labels as the determiner of their color. **(P3-2: 10 points)**

In [25]:
pca = PCA(n_components=3)
pca.fit(selected_images)
selected_images = pca.transform(selected_images)
selected_images = np.real(selected_images)

In [26]:
df_transformed = pd.DataFrame(data=selected_images, columns=['component_1', 'component_2', 'component_3'])
df_transformed['label'] = selected_labels

fig = px.scatter_3d(df_transformed, x='component_1', y='component_2', z='component_3', color='label')
fig.show()

# Sec 4: K-Means

Using K-means, we want to perform clustering on the 3D samples. Consider 4 clusters. After running the K-means algorithm on the data, determine to which cluster each sample belongs. Then, use `plotly` to plot the 3D samples colored with respect to their clusters. **(P4-1: 15 points)**

In [28]:
class KMeans:
    """
    Fits it on data, then uses predict to get cluster labels.
    """

    def __init__(self, n_clusters=4):
        """
        YOUR CODE (IF NECESSARY)
        """
        self.centroids = None
        self.n_clusters = n_clusters

    def fit(self, x, y=None):
        """Fits the training data
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Training samples
        y : array-like, shape = [n_samples, n_target_values]
            Target values
        Returns
        -------
        self : object
        """

        n_samples, n_features = x.shape
        """
        YOUR CODE
        """
        self.centroids = None
        random_indices = np.random.choice(n_samples, self.n_clusters, replace=False)
        self.centroids = x[random_indices]
        while True:
            y_pred = self.predict(x)
            new_centroids = np.zeros((self.n_clusters, n_features))
            for i in range(self.n_clusters):
                new_centroids[i] = np.mean(x[y_pred == i], axis=0)
            if np.allclose(self.centroids, new_centroids):
                break
            self.centroids = new_centroids

        return self

    def predict(self, x):
        """ Predicts the cluster label
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Test samples
        Returns
        -------
        Predicted cluster label
        """

        y_pred = None
        """
        YOUR CODE
        """
        y_pred = np.zeros(len(x))
        for i in range(len(x)):
            distances = np.linalg.norm(self.centroids - x[i], axis=1)
            y_pred[i] = np.argmin(distances)
        return y_pred

In [29]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(selected_images)
y_pred = kmeans.predict(selected_images)
df_transformed['cluster'] = y_pred
fig = px.scatter_3d(df_transformed, x='component_1', y='component_2', z='component_3', color='cluster')
fig.show()

Measure the quality of the clustering using "silhoutte score" **(P4-2: 5 points)**

Note: You can use `sklearn` or other packages to calculate the score.

In [38]:
from sklearn.metrics import silhouette_score
print(f'Silhouette Score with predicted labels: {silhouette_score(selected_images, y_pred)}')

Silhouette Score with predicted labels: 0.3759608975144656


Compare the result of the clustering with the original labels. To be more precise, in this part, you should clearly answer the following questions with details:<br>

- Does each cluster represent a true label?
- What is your conclusion?

**(P4-3: 10 points)**

In [36]:
print(f'Silhouette Score with original labels: {silhouette_score(selected_images, selected_labels)}')

Silhouette Score with original labels: 0.22307026883212527


As it can be seen the Silhouette Score of the original labels is not good as well. It means that each cluster does not necessarily represent a true label. What k-means does is clustering data points based on their feature similarity and their distance to a centroid without considering their true labels.  
My conclusion is that if data is capable of being clustered by k-means, it eventually will converge to an acceptable clustering. But if the data has some greater complexity underlying that can't be captured by k-means, no matter how many iterations you run the algorithm, the quality of clustering won't get much better.

# Sec 5: Neural Net (Optional with Bonus Points)

Reshape the images so that they become of shape `64x64` **(P5-1: 5 points)**

In [39]:
x = x.reshape(-1, 64, 64)

Use different types of augmentation techniques (such as Random Flipping, Random Rotation, etc.) to increase the size of the training dataset. **(P5-2: 10 points)**

In [43]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

labels = np.array(df['label'].values)
data_gen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

extended_x = []
extended_y = []
for i in range(len(x)):
    image = x[i]
    # Add an additional dimension if the image is grayscale
    if len(image.shape) < 3:
        image = np.expand_dims(image, axis=-1)
    label = labels[i]
    extended_x.append(image)
    extended_y.append(label)
    for j in range(5):
        new_image = data_gen.random_transform(image)
        extended_x.append(new_image)
        extended_y.append(label)
extended_x = np.array(extended_x)
extended_y = np.array(extended_y)

Using the augmented training set, train a neural network consisting of 2D convolutional layers. The exact architecture of the network is arbitrary. However, the last layer should have the "softmax" function as its activation. Also, using "max pooling" layers after convolutional ones is advised.

Note: Don't get frustrated if your network does not appear to be a good one. Give it some time to be trained. For instance, you might need to train it for 100 epochs. Make sure to try that out before quitting!

**(P5-3: 20 points)**

In [46]:
label_encoder = LabelEncoder()
label_encoder.fit(extended_y)
encoded_y = label_encoder.transform(extended_y)

In [58]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
train_x, test_x, train_y, test_y = train_test_split(extended_x, encoded_y, test_size=0.2, random_state=42)
test_x, validation_x, test_y, validation_y = train_test_split(test_x, test_y, test_size=0.5, random_state=41)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(39, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=50, validation_data=(validation_x, validation_y))


Epoch 1/50
60/60 [==============================] - 20s 280ms/step - loss: 3.6646 - accuracy: 0.0432 - val_loss: 3.6591 - val_accuracy: 0.0708
Epoch 2/50
60/60 [==============================] - 17s 278ms/step - loss: 3.6599 - accuracy: 0.0443 - val_loss: 3.6566 - val_accuracy: 0.0708
Epoch 3/50
60/60 [==============================] - 16s 276ms/step - loss: 3.6599 - accuracy: 0.0474 - val_loss: 3.6522 - val_accuracy: 0.0708
Epoch 4/50
60/60 [==============================] - 17s 277ms/step - loss: 3.6586 - accuracy: 0.0464 - val_loss: 3.6496 - val_accuracy: 0.0708
Epoch 5/50
60/60 [==============================] - 16s 274ms/step - loss: 3.6013 - accuracy: 0.0495 - val_loss: 3.4576 - val_accuracy: 0.0833
Epoch 6/50
60/60 [==============================] - 19s 320ms/step - loss: 3.3141 - accuracy: 0.0828 - val_loss: 3.0055 - val_accuracy: 0.2083
Epoch 7/50
60/60 [==============================] - 16s 269ms/step - loss: 2.8905 - accuracy: 0.1646 - val_loss: 2.4014 - val_accuracy: 0.2792

Measure the accuracy of the network on the validation and testing datasets **(P5-4: 5 points)**

In [59]:
val_loss, val_acc = model.evaluate(validation_x, validation_y)
test_loss, test_acc = model.evaluate(test_x, test_y)
print("Validation accuracy:", val_acc)
print("Test accuracy:", test_acc)

8/8 [==============================] - 1s 102ms/step - loss: 0.1726 - accuracy: 0.9583
Validation accuracy: 0.9333333373069763
Test accuracy: 0.9583333134651184
